https://colab.research.google.com/drive/17IVyNAIr19dfdIog6CLtHMdVBfqj7wVs

In [0]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
from keras.applications import vgg19
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
import tensorflow as tf
import keras

In [0]:
!wget https://www.dropbox.com/s/aqdoilzpclo59il/style%20image.png
!wget https://www.dropbox.com/s/h7b1hn6ck4xmeuf/cat.png 

In [0]:
import cv2

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
base_img = cv2.imread('/content/cat.png')
base_img = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGB)
base_img = cv2.resize(base_img,(224,224))
plt.imshow(base_img)
plt.axis('off')

In [0]:
style_img = cv2.imread('/content/style image.png')
style_img = cv2.cvtColor(style_img, cv2.COLOR_BGR2RGB)
style_img = cv2.resize(style_img,(224,224))
plt.imshow(style_img)
plt.axis('off')

In [0]:
from keras.applications import vgg19

In [0]:
K.set_learning_phase(0)
model = vgg19.VGG19(include_top=False, weights='imagenet')

In [0]:
model.summary()

In [0]:
base_img = base_img.reshape(1,224,224,3)/255

In [0]:
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.get_layer('block3_conv4').output])
layer_output_base = get_3rd_layer_output([base_img])[0]

In [0]:
layer_output_base.shape

In [0]:
layer_contributions_content = {
'block3_conv4': 0.1
}

In [0]:
layer_contributions_style = {
    'block1_pool':1,
   'block2_pool':1,
'block3_conv4': 1,
}

In [0]:
img_nrows = 224
img_ncols = 224

In [0]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (pow(channels,2)) * (pow(size,2)))

In [0]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)
for layer_name in layer_contributions_content:
  coeff = layer_contributions_content[layer_name]
  activation = layer_dict[layer_name].output
  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
  loss += coeff * K.sum(K.square(activation - layer_output_base)) / scaling
  
for layer_name in layer_contributions_style:
  coeff = layer_contributions_style[layer_name]
  activation = layer_dict[layer_name].output
  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
  style_layer_output = K.function([model.layers[0].input],[model.get_layer(layer_name).output])
  layer_output_style = style_layer_output([style_img.reshape(1,224,224,3)/255])[0][0]
  loss += style_loss(layer_output_style, activation[0])

In [0]:
dream = model.input
grads = K.gradients(loss, dream)[0]
print(dream)
print(grads)

In [0]:
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

In [0]:
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

In [0]:
def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_values = outs[1]
  return loss_value, grad_values

In [0]:
def gradient_ascent(x, iterations, step, max_loss=None):
  for i in range(iterations):
    loss_value, grad_values = eval_loss_and_grads(x)
    print('...Loss value at', i, ':', loss_value)
    x -= step * grad_values
  return x

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
base_img.shape

In [0]:
plt.imshow(base_img[0,:,:,:])

In [0]:
img = base_img.copy().reshape(1,224,224,3).astype('float32')

In [0]:
for i in range(2000):
  #print(loss)
  step=0.001
  loss_value, grad_values = eval_loss_and_grads(img)
  print('...Loss value at', i, ':', loss_value)
  img -= step * grad_values
  if(i%100 ==0):
    img2 = img.copy().reshape(224,224,3)
    img2 = np.clip(img2*255, 0, 255).astype('uint8')
    plt.imshow(img2)
    plt.axis('off')
    plt.show()